# Style encoding as by Xander Steenbrugge
https://www.youtube.com/watch?v=dCKbRCUyop8

#### the faces' latent space is created with pbaylies' awesome StyleGAN encoder (building on original work from Puzer): 

pbaylies repo: https://github.com/pbaylies/stylegan-encoder
StyleGAN paper: https://arxiv.org/abs/1812.04948

#### the latent space can then be navigated with tr1pzz's fork of the original ShenYujun's InterFaceGAN

Original Repo: https://github.com/ShenYujun/InterFaceGAN 
Paper: https://arxiv.org/abs/1907.10786

## Part I: Encoding images into StyleGAN's latent space

Runtime type must be GPU enabled

In [0]:
import os
import numpy as np
import humanize
import psutil
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
printm() 

In [0]:
import tensorflow as tf
tf.VERSION
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [0]:
!nvcc --version

In [0]:
!pwd
!ls

## config globals

In [0]:
autoMode = 1 # auto run mode - get uri[uri.rfind("/")+1:] files from imgsToAutodownload
uploadRawImages = 0 # manually upload raw images
takePhotoImages = 0 # take photo for raw images

imgsToAutodownload = [
  "https://github.com/sifbuilder/eodoes/raw/master/packages/eodoes-eodo-eofaces/img/2019-09-22_19-04-13.jpg"
#  , "https://github.com/sifbuilder/eodoes/raw/master/packages/eodoes-eodo-eofaces/img/2019-09-22_19-09-43.jpg"
]

dodownload = 1 # download results

emptyspace = 0 # or reset runtime
toSmptyFolders = ["stylegan-encoder", "InterFaceGAN", "output_vectors.npy"]


In [0]:
if emptyspace == 1:
  print("remove folders %s" %toSmptyFolders)
  for folder in toSmptyFolders:
    print("folder: %s" %folder)
    !rm -r $folder


### Clone pbaylies' stylegan encoder:

In [0]:
path = 'stylegan-encoder'
isExist = os.path.exists(path)
if not isExist:
  print("stylegan-encoder does not exist. will clone")
  !git clone https://github.com/pbaylies/stylegan-encoder
else:
  print("stylegan-encoder already exists. will not clone")

### cd into the encoder repo folder and create image folders:

In [0]:
cd stylegan-encoder

In [0]:
mkdir -p aligned_images

In [0]:
mkdir -p raw_images

In [0]:
ls

### Get the images:

*   Use HD images (preferably > 1000x1000 pixels)
*   Start with 3 - 6 different images
*   Make sure faces are not too small
*   Neutral expressions & front facing faces will be better
*   Clear, uniform lighting conditions are also recommened



In [0]:
# Opt 1: get the images from the imgsToAutodownload global array set above 
if autoMode == 1:
  for uri in imgsToAutodownload:
    print("uri: %s" %uri)
    !wget $uri
    fname=uri[uri.rfind("/")+1:]
    print("fname: %s" %fname)            
    !mv $fname raw_images/


In [0]:
# Opt 2: upload images from the file system
if uploadRawImages == 1:
  print("upload raw images and move to raw_images")
  from google.colab import files
  uploaded = files.upload()
  for fname in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(name=fname, length=len(uploaded[fname])))
    !mv $fname 'raw_images/'


In [0]:
# Opt 3: take image from the web cam
if takePhotoImages == 1:
  from base64 import b64decode
  from IPython.display import HTML, Audio
  from google.colab.output import eval_js
  from PIL import Image
  from datetime import datetime

  VIDEO_HTML = """
  <video autoplay
   width=%d height=%d style='cursor: pointer;'></video>
  <script>

  var video = document.querySelector('video')

  navigator.mediaDevices.getUserMedia({ video: true })
    .then(stream=> video.srcObject = stream)

  var data = new Promise(resolve=>{
    video.onclick = ()=>{
      var canvas = document.createElement('canvas')
      var [w,h] = [video.offsetWidth, video.offsetHeight]
      canvas.width = w
      canvas.height = h
      canvas.getContext('2d')
            .drawImage(video, 0, 0, w, h)
      video.srcObject.getVideoTracks()[0].stop()
      video.replaceWith(canvas)
      resolve(canvas.toDataURL('image/jpeg', %f))
    }
  })
  </script>
  """

  def take_photo(quality=1.0, size=(800,600)):
    display(HTML(VIDEO_HTML % (size[0],size[1],quality)))
    data = eval_js("data")
    binary = b64decode(data.split(',')[1])
    f = io.BytesIO(binary)
    img = np.asarray(Image.open(f))

    timestampStr = datetime.now().strftime("%d-%b-%Y (%H:%M:%S.%f)")
    filename = 'raw_images/photo_%s.jpeg' %timestampStr
    Image.fromarray(img).save(filename)
    print('Image captured and saved to %s' %filename)

  img = take_photo() # click the image to capture a frame!

### List the contents of our image folder:
images can be deleted by right clicking on them in the file tab


In [0]:
import os
from PIL import Image

imgs = sorted(os.listdir('raw_images'))

print("Found %d images in %s" %(len(imgs), 'raw_images'))
if len(imgs) == 0:
  print("No image found !!!")
else:
  print(imgs)

  for img_name in imgs:
    img_path = 'raw_images/' + img_name
    print("img_path: %s" %img_path)
    img = Image.open(img_path)
    w,h = img.size
    rescale_ratio = 256 / min(w,h)
    img = img.resize((int(rescale_ratio*w),int(rescale_ratio*h)), Image.LANCZOS)
    display(img)

### auto-align faces:
1.   Look for faces in the images
2.   Crop out the faces from the images
3.   Align the faces (center the nose and make the eyes horizontal)
4.   Rescale the resulting images and save them in "aligned_images" folder



In [0]:
!python align_images.py raw_images/ aligned_images/ --output_size=1048

### display the aligned images:

In [0]:
def display_folder_content(folder, res = 256):
  if folder[-1] != '/': folder += '/'
  for i, img_path in enumerate(sorted(os.listdir(folder))):
    if '.png' in img_path:
      display(Image.open(folder+img_path).resize((res,res)), 'img %d: %s' %(i, img_path))
      print('\n')
      
display_folder_content('aligned_images')

### Manually clean the 'aligned_images' directory

> #### 1. Manually remove all 'bad' images that do not look sharp or clear 
> #### 2. Make sure not to have too many faces (preferably 8 max)




### Download a pretrained resnet encoder: (see video for what this does)
### --> This model takes an image as input and estimates the corresponding latent code

In [0]:
!gdown https://drive.google.com/uc?id=1aT59NFy9-bNyXjDuZOTMl0qX0jmZc6Zb
!mkdir -p data
!mv finetuned_resnet.h5 data
!rm -rf generated_images latent_representations

### encode the images
(adapt the encoding params: they determine the latent representations & images!)
see extra encoding options: https://github.com/pbaylies/stylegan-encoder/blob/master/encode_images.py

Note: This script will also download:
*   The pretrained StyleGAN network from NVIDIA trained on faces
*   A pretrained VGG-16 network, trained on ImageNet

After guessing the initial latent codes using the pretrained ResNet, it will run gradient descent to optimize the latent faces!

Note that by default, we're optimizing w vectors, not z-vectors!


In [0]:
print("aligned_images contains %d images ready for encoding!" %len(os.listdir('aligned_images/')))

In [0]:
rm -f videos/* # clean up the videos-to-generate folder

In [0]:
# Set the batch size to the number of images in the aligned_images/ folder
# If the results don't look great: Play with the encoding arguments!!!
# > 1. More iterations             eg.  1000 (def: 200))
# > 2. Decrease the L1 penalty to  eg.   0.1 (def: 0.3))
# > 3. Lower initial learning rate eg. 0.005 (def: 0.01) 
#      Decay_rate (def: 0.8)
# > 4. Find out about the other encoding options here: 
# https://github.com/pbaylies/stylegan-encoder/blob/master/encode_images.py 
    
!python encode_images.py \
--batch_size=2 \
--output_video=True \
--load_resnet='data/finetuned_resnet.h5' \
--lr=0.01 \
--decay_rate=0.8 \
--iterations=100 \
--use_l1_penalty=0.3 \
aligned_images/ generated_images/ latent_representations/

print("\n************ Latent code optimization finished! ***************")

#### load the StyleGAN network into memory:

In [0]:
import dnnlib, pickle
import dnnlib.tflib as tflib
tflib.init_tf()
synthesis_kwargs = dict(output_transform=dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True), minibatch_size=1)

model_dir = 'cache/'
model_path = [model_dir+f for f in os.listdir(model_dir) if 'stylegan-ffhq' in f][0]
print("Loading StyleGAN model from %s..." %model_path)

with dnnlib.util.open_url(model_path) as f:
  generator_network, discriminator_network, averaged_generator_network = pickle.load(f)
  
print("StyleGAN loaded & ready for sampling!")

In [0]:
def generate_images(generator, latent_vector, z = True):
    batch_size = latent_vector.shape[0]
    
    if z: #Start from z: run the full generator network
        return generator.run(latent_vector.reshape((batch_size, 512)), None, randomize_noise=False, **synthesis_kwargs)
    else: #Start from w: skip the mapping network
        return generator.components.synthesis.run(latent_vector.reshape((batch_size, 18, 512)), randomize_noise=False, **synthesis_kwargs)

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline

def plot_imgs(model, rows, columns):
  for i in range(rows):
    f, axarr = plt.subplots(1,columns, figsize = (20,8))
    for j in range(columns):
      img = generate_images(model, np.random.randn(1,512), z = True)[0]
      axarr[j].imshow(img)
      axarr[j].axis('off')
      axarr[j].set_title('Resolution: %s' %str(img.shape))
    plt.show()

#### show the encoded images

If the results don't look great: Play with the encoding arguments!!!
> 1. Run the optimization for more iterations (eg 1000)
> 2. Decrease the L1 penalty (to eg 0.1)
> 3. Try a lower initial learning rate (eg 0.005) or play with the decay_rate
> 4. Find out about the other encoding options here: https://github.com/pbaylies/stylegan-encoder/blob/master/encode_images.py

In [0]:
import numpy as np

for f in sorted(os.listdir('latent_representations')):
  w = np.load('latent_representations/' + f).reshape((1,18,-1))
  img = generate_images(averaged_generator_network, w, z = False)[0]
  plt.imshow(img)
  plt.axis('off')
  plt.title("Generated image from %s" %f)
  plt.show()

#### show the encoded samples side by side with the original ones:

In [0]:
import matplotlib.pyplot as plt

def plot_two_images(img1,img2, img_id, fs = 12):
  f, axarr = plt.subplots(1,2, figsize=(fs,fs))
  axarr[0].imshow(img1)
  axarr[0].title.set_text('Encoded img %d' %img_id)
  axarr[1].imshow(img2)
  axarr[1].title.set_text('Original img %d' %img_id)
  plt.setp(plt.gcf().get_axes(), xticks=[], yticks=[])
  plt.show()

def display_sbs(folder1, folder2, res = 256):
  if folder1[-1] != '/': folder1 += '/'
  if folder2[-1] != '/': folder2 += '/'
    
  imgs1 = sorted([f for f in os.listdir(folder1) if '.png' in f])
  imgs2 = sorted([f for f in os.listdir(folder2) if '.png' in f])
  if len(imgs1)!=len(imgs2):
    print("Found different amount of images in aligned vs raw image directories. That's not supposed to happen...")
  
  for i in range(len(imgs1)):
    img1 = Image.open(folder1+imgs1[i]).resize((res,res))
    img2 = Image.open(folder2+imgs2[i]).resize((res,res))
    plot_two_images(img1,img2, i)
    print("")
     
display_sbs('generated_images/', 'aligned_images/', res = 512)

#### display videos of image generation 

In [0]:
ls videos

In [0]:
videos_dir = 'videos'  
videos_dir_bck = 'videos'  


!ls $videos_dir

thereisbck = os.path.isdir(videos_dir_bck)
print("thereisbck %s" %(thereisbck))

if thereisbck == True:
  print("video folders with first names: %s" %(videos_dir_bck))
else:
  print("copy to bck")
  !cp -r $videos_dir $videos_dir_bck

vids = sorted(os.listdir(videos_dir))
print("Found %d videos in %s" %(len(vids), videos_dir))
for i, vid_fullname in enumerate(vids):
  vid_name = vid_fullname.split('.')[0]
  vid_ext = vid_fullname.split('.')[1]
  newvid_fullname = 'vid_%02d.%s' %(i, vid_ext)
  os.rename(videos_dir + '/' + vid_fullname, videos_dir + '/' + newvid_fullname)
!ls $videos_dir

In [0]:
from moviepy.editor import *
from moviepy.video.io.VideoFileClip import VideoFileClip
from moviepy.Clip import Clip
from IPython.display import display

videos_dir = 'videos'  
vids = sorted(os.listdir(videos_dir))
for i, vid in enumerate(vids):
  vid_name = videos_dir + '/' + vid
  print("vid %d : %s" %(i, vid_name))
  clip = VideoFileClip(vid_name)
  display(clip.ipython_display(height=512, autoplay=1, loop=1))


### set the good images and dump their latent vectors to the root folder on disk 



In [0]:
folder = 'latent_representations'

good_images = []
for i, latents in enumerate(sorted(os.listdir(folder))):
  good_images.append(i)
      
# good_images = [0,1,2,3]  # Uncomment and Change these numbers to pick out latents that worked well (see the image plots)

In [0]:
out_file = '../output_vectors.npy'

import numpy as np
latents = sorted(os.listdir('latent_representations'))

final_w_vectors = []
for img_id in good_images:
  w = np.load('latent_representations/' + latents[img_id])
  final_w_vectors.append(w)

final_w_vectors = np.array(final_w_vectors)
np.save(out_file, final_w_vectors)
print("%d latent vectors of shape %s saved to %s!" %(len(good_images), str(w.shape), out_file))

In [0]:
# up to root folder
%cd ..

# Reset runtime

In [0]:
import os

# os.kill(os.getpid(), 9)

from numba import cuda
cuda.select_device(0)
cuda.close()

print("cpu_ram_total: %s" %psutil.virtual_memory().total)
print("cpu_ram_avail: %s" %psutil.virtual_memory().available)

# Notebook II: navigate through latent space

In [0]:
!pwd
!ls

In [0]:
import tensorflow as tf
tf.VERSION
!nvcc --version

#### Clone tr1pzz's fork of original ShenYujun's InterFaceGAN


In [0]:
import os
path = 'InterFaceGAN'
facegangit = "https://github.com/tr1pzz/InterFaceGAN.git"
# facegangit = "https://github.com/ShenYujun/InterFaceGAN.git"
isExist = os.path.exists(path)
if not isExist:
  print("InterFaceGAN does not exist. clone %s" %facegangit)
  !git clone $facegangit
else:
  print("InterFaceGAN already exists. will not clone")

In [0]:
# move latent vectors to gan folder
!mv output_vectors.npy InterFaceGAN

In [0]:
cd InterFaceGAN/

#### Download the pretrained StyleGAN FFHQ network from NVIDIA:

In [0]:
!gdown https://drive.google.com/uc?id=1MEGjdvVpUsu1jB4zrXZN7Y4kBBOzizDQ

In [0]:
!mv karras2019stylegan-ffhq-1024x1024.pkl models/pretrain/karras2019stylegan-ffhq-1024x1024.pkl

#### load the latent space vectors from stylegan


In [0]:
import numpy as np
final_w_vectors = np.load('output_vectors.npy')

print("%d latent vectors of shape %s loaded from %s!" %(final_w_vectors.shape[0], str(final_w_vectors.shape[1:]), 'output_vectors.npy'))

#### configure the latent-space interpolation to morph the faces:

the InterFaceGAN includes pretrained latent directions (age, eyeglasses, gender, pose, smile). others can be trained. see boundaries at: https://github.com/ShenYujun/InterFaceGAN/tree/master/boundaries

In [0]:
latent_direction = 'age'     #### Pick one of ['age', 'eyeglasses', 'gender', 'pose', 'smile']
morph_strength_start = -3           # Controls how strongly we push the face into a certain latent direction (try 1-5)
morph_strength_end = 1           # Controls how strongly we push the face into a certain latent direction (try 1-5)
nr_interpolation_steps = 24  # The amount of intermediate steps/frames to render along the interpolation path


#### generate the latent interpolation


In [0]:
boundary_file = 'stylegan_ffhq_%s_w_boundary.npy' %latent_direction
print("Dump Latent interpolation to disk!")
resultsPath = "results/%s" %latent_direction # eg. results/age
!rm -r $resultsPath

boundary = "boundaries/stylegan_ffhq_%s_w_boundary.npy" %latent_direction
startDistance = "%.2f" %morph_strength_start
endDistance = "%.2f" %morph_strength_end
steps = "%d" %nr_interpolation_steps

print("resultsPath: %s" %resultsPath)
print("latent_direction: %s" %latent_direction)
print("boundary: %s" %boundary)
print("startDistance: %s" %startDistance)
print("endDistance: %s" %endDistance)
print("steps: %s" %steps)


!python edit.py \
      -m stylegan_ffhq \
      -b $boundary \
      -s Wp \
      -i 'output_vectors.npy' \
      -o $resultsPath \
      --start_distance $startDistance \
      --end_distance $endDistance \
      --steps=$steps

In [0]:
boundary_file = 'stylegan_ffhq_%s_w_boundary.npy' %latent_direction
print("Dump Latent interpolation to disk!")
resultsPath = "results/%s" %latent_direction # eg. results/age
!rm -r $resultsPath

boundary = "boundaries/stylegan_ffhq_%s_w_boundary.npy" %latent_direction
startDistance = "%.2f" %morph_strength_start
endDistance = "%.2f" %morph_strength_end
steps = "%d" %nr_interpolation_steps

print("resultsPath: %s" %resultsPath)
print("latent_direction: %s" %latent_direction)
print("boundary: %s" %boundary)
print("startDistance: %s" %startDistance)
print("endDistance: %s" %endDistance)
print("steps: %s" %steps)


!python edit.py \
      -m stylegan_ffhq \
      -b $boundary \
      -s Wp \
      -i 'output_vectors.npy' \
      -o $resultsPath \
      --start_distance $startDistance \
      --end_distance $endDistance \
      --steps=$steps

#### configure video output

In [0]:
image_folder = 'results/%s' %latent_direction
print ("images folder %s" %image_folder)
video_fps = 12.

#### render the videos

In [0]:
from moviepy.editor import *
import cv2

out_path = 'output_videos/'

images = [img_path for img_path in sorted(os.listdir(image_folder)) if '.jpg' in img_path]
os.makedirs(out_path, exist_ok=True)

prev_id = None
img_sets = []
for img_path in images:
  img_id = img_path.split('_')[0]
  if img_id == prev_id: #append
    img_sets[-1].append(img_path)
    
  else: #start a new img set
    img_sets.append([])
    img_sets[-1].append(img_path)
  prev_id = img_id

print("Found %d image sets!\n" %len(img_sets))
if image_folder[-1] != '/':
  image_folder += '/'

def make_video(images, vid_name):
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video = cv2.VideoWriter(vid_name, fourcc, video_fps, (1024, 1024))
    gen = {}
    for img in images:
      video.write(img)
    video.release()
    print('finished '+ vid_name)  
    
for i in range(len(img_sets)):
  print("\nGenerating video %d..." %i)
  set_images = []
  vid_name = out_path + 'out_video_%s_%02d.mp4' %(latent_direction,i)
  for img_path in img_sets[i]:
    set_images.append(cv2.imread(image_folder + img_path))

  set_images.extend(reversed(set_images))
  make_video(set_images, vid_name)

#### display the resulting videos inside this Notebook

In [0]:
vids = sorted([f for f in os.listdir('output_videos')])
print ("vids %s" %vids)
for i, vid_name in enumerate(vids):
  vid_path = 'output_videos/%s' %vid_name
  print ("vid_path %s" %vid_path)
  clip = VideoFileClip(vid_path)
  display(clip.ipython_display(height=512, autoplay=1, loop=1))

#### Some things to try:
* blend between two faces by doing a linear interpolation in the latent space
* the StyleGAN vector has 18x512 dimensions, each of those 18 going into a different layer of the generator ... take the first 9 from person A and the next 9 from person B
(This is why it's called "Style-GAN": you can manipulate the style of an image at multiple levels of the Generator!)
* interpolate in Z-space rather than in W-space (see InterFaceGan paper & repo)

#### share in the comments of the vieo https://www.youtube.com/watch?v=dCKbRCUyop8 
#### or tag the author on Twitter @xsteenbrugge: https://twitter.com/xsteenbrugge